# 第4章 朴素贝叶斯

1．朴素贝叶斯法是典型的生成学习方法。生成方法由训练数据学习联合概率分布
$P(X,Y)$，然后求得后验概率分布$P(Y|X)$。具体来说，利用训练数据学习$P(X|Y)$和$P(Y)$的估计，得到联合概率分布：

$$P(X,Y)＝P(Y)P(X|Y)$$

概率估计方法可以是极大似然估计或贝叶斯估计。

2．朴素贝叶斯法的基本假设是条件独立性，

$$\begin{aligned} P(X&=x | Y=c_{k} )=P\left(X^{(1)}=x^{(1)}, \cdots, X^{(n)}=x^{(n)} | Y=c_{k}\right) \\ &=\prod_{j=1}^{n} P\left(X^{(j)}=x^{(j)} | Y=c_{k}\right) \end{aligned}$$


这是一个较强的假设。由于这一假设，模型包含的条件概率的数量大为减少，朴素贝叶斯法的学习与预测大为简化。因而朴素贝叶斯法高效，且易于实现。其缺点是分类的性能不一定很高。

3．朴素贝叶斯法利用贝叶斯定理与学到的联合概率模型进行分类预测。

$$P(Y | X)=\frac{P(X, Y)}{P(X)}=\frac{P(Y) P(X | Y)}{\sum_{Y} P(Y) P(X | Y)}$$
 
将输入$x$分到后验概率最大的类$y$。

$$y=\arg \max _{c_{k}} P\left(Y=c_{k}\right) \prod_{j=1}^{n} P\left(X_{j}=x^{(j)} | Y=c_{k}\right)$$

后验概率最大等价于0-1损失函数时的期望风险最小化。


模型：

- 高斯模型
- 多项式模型
- 伯努利模型

In [ ]:
import numpy as np # 导入numpy
import pandas as pd # 导入pandas
import matplotlib.pyplot as plt # 导入画图
%matplotlib inline

from sklearn.datasets import load_iris # 导入数据集
from sklearn.model_selection import train_test_split # 导入数据集分割方法

from collections import Counter # 导入Counter，用于统计字典key的数量
import math # 导入数学包

In [ ]:
# data
def create_data():
    iris = load_iris() # 加载数据集
    df = pd.DataFrame(iris.data, columns=iris.feature_names) # 创建表格对象，放入X
    df['label'] = iris.target # 增加一列，放入y
    df.columns = [
        'sepal length', 'sepal width', 'petal length', 'petal width', 'label'
    ] # 列名
    data = np.array(df.iloc[:100, :]) # 取前100行，所有列，创建一个100*5的矩阵
    # print(data)
    return data[:, :-1], data[:, -1] # 返回X和y

In [ ]:
X, y = create_data() # 创建数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 分割数据集，测试集占30%
# print(X_train)

In [ ]:
X_test[0], y_test[0] # 打印测试集0行

参考：https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/

## GaussianNB 高斯朴素贝叶斯

特征的可能性被假设为高斯

概率密度函数：
$$P(x_i | y_k)=\frac{1}{\sqrt{2\pi\sigma^2_{yk}}}exp(-\frac{(x_i-\mu_{yk})^2}{2\sigma^2_{yk}})$$

数学期望(mean)：$\mu$

方差：$\sigma^2=\frac{\sum(X-\mu)^2}{N}$

In [ ]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    # 数学期望
    @staticmethod
    def mean(X):
        # return sum(X) / float(len(X)) # 返回数学期望
        # 改写一下，方便debug
        X_sum = sum(X)
        X_length = float(len(X))
        return X_sum / X_length

    # 标准差（方差）
    def stdev(self, X):
        avg = self.mean(X) # 计算均值
        # return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X))) # TypeError: only size-1 arrays can be converted to Python scalars
        # return np.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X))) # 改用numpy的sqrt函数
        # 改写一下
        x_pow_sum = 0.
        for x in X:
            x_pow = pow(x - avg, 2)
            x_pow_sum += x_pow
        d = x_pow_sum / float(len(X)) # 总体方差
        return np.sqrt(d) # 标准差



    # 概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2)))) # 指数项
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent # 返回的就是上面那个概率密度函数的计算结果

    # 处理X_train
    def summarize(self, train_data):
        # *参数用于解包tuple对象的每个元素，作为一个一个的位置参数传入到函数中
        # **参数用于解包dict对象的每个元素，作为一个一个的关键字参数(以key=value形式)传入到函数中
        # summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        # 改写一下
        train_data_zip = zip(*train_data)
        summaries = []
        for i in train_data_zip: # train_data是一个36*4的矩阵，转成zip后，每个i是长度为36的数组，即为矩阵的某一列
            o_tuple = (self.mean(i), self.stdev(i)) # 计算均值和总体标准差
            summaries.append(o_tuple) # 结果放入list中
        return summaries # 返回

    # 分类别求出数学期望和标准差
    def fit(self, X, y):
        labels = list(set(y)) # 利用set去重，再转为list
        data = {label: [] for label in labels} # data为一个对象，key有两个，为1.或0.，value为数组
        X_and_y_zip = zip(X, y)
        for f, label in X_and_y_zip: # 此循环用于将X分类
            data[label].append(f) # 对应y_j放入X_i
        # self.model = {
        #     label: self.summarize(value)
        #     for label, value in data.items()
        # }
        # 改写一下
        self.model = {}
        for label, value in data.items(): # 遍历所有项
            self.model[label] = self.summarize(value) # 对应label将summarize结果放入model对象中
        return 'gaussianNB train done!'

    # 计算概率
    def calculate_probabilities(self, input_data):
        # summaries:{0.0: [(5.0, 0.37),(3.42, 0.40)], 1.0: [(5.8, 0.449),(2.7, 0.27)]}
        # input_data:[1.1, 2.2]
        probabilities = {}
        for label, value in self.model.items(): # 遍历所有label
            probabilities[label] = 1 # 方便做乘法
            for i in range(len(value)): # 遍历某个label的所有期望和标准差
                mean, stdev = value[i] # 取出一个期望和标准差
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev) # 代入概率密度计算概率，结果对应label累乘
        return probabilities # 返回结果

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        # label = sorted(
        #     self.calculate_probabilities(X_test).items(),
        #     key=lambda x: x[-1])[-1][0] # 按照概率结果排序，取最大的
        # 改写一下
        items = self.calculate_probabilities(X_test).items() # 计算测试集的概率，并取出所有项
        sorted1 = sorted(items, key=lambda x: x[-1]) # 参数1，要排序的对象；参数2，使用匿名函数lambda，指定根据items中的概率结果排序；默认升序
        label = sorted1[-1][0] # 取最大值的label
        return label # 返回

    # 计算得分
    def score(self, X_test, y_test):
        right = 0 # 记录分类正确的数量
        for X, y in zip(X_test, y_test): # 遍历测试集
            label = self.predict(X) # 对每个对象进行分类
            if label == y: # 判断结果
                right += 1 # 分类正确

        return right / float(len(X_test)) # 返回得分

In [ ]:
model = NaiveBayes() # 实例化对象

In [ ]:
model.fit(X_train, y_train) # 得到训练集的方差和标准差

In [ ]:
print(model.predict([4.4,  3.2,  1.3,  0.2])) # 随便找个例子分类一下试试

In [ ]:
model.score(X_test, y_test) # 测试，计算得分

### scikit-learn实例

In [ ]:
from sklearn.naive_bayes import GaussianNB # 导入高斯朴素贝叶斯

In [ ]:
clf = GaussianNB() # 实例化对象
clf.fit(X_train, y_train) # 训练

In [ ]:
clf.score(X_test, y_test) # 测试

In [ ]:
clf.predict([[4.4,  3.2,  1.3,  0.2]]) # 随便一个测试一下

In [ ]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB # 伯努利模型和多项式模型

## 第4章朴素贝叶斯法-习题

### 习题4.1
&emsp;&emsp;用极大似然估计法推出朴素贝叶斯法中的概率估计公式(4.8)及公式 (4.9)。

**解答：**  
**第1步：**证明公式(4.8)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$  
由于朴素贝叶斯法假设$Y$是定义在输出空间$\mathcal{Y}$上的随机变量，因此可以定义$P(Y=c_k)$概率为$p$。  
令$\displaystyle m=\sum_{i=1}^NI(y_i=c_k)$，得出似然函数：$$L(p)=f_D(y_1,y_2,\cdots,y_n|\theta)=\binom{N}{m}p^m(1-p)^{(N-m)}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{N}{m}\left[mp^{(m-1)}(1-p)^{(N-m)}-(N-m)p^m(1-p)^{(N-m-1)}\right] \\
& = \binom{N}{m}\left[p^{(m-1)}(1-p)^{(N-m-1)}(m-Np)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{m}{N}$  
显然$\displaystyle P(Y=c_k)=p=\frac{m}{N}=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k)}{N}$，公式(4.8)得证。

----

**第2步：**证明公式(4.9)：$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$  
令$P(X^{(j)}=a_{jl}|Y=c_k)=p$，令$\displaystyle m=\sum_{i=1}^N I(y_i=c_k), q=\sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)$，得出似然函数：$$L(p)=\binom{m}{q}p^q(i-p)^{m-q}$$使用微分求极值，两边同时对$p$求微分：$$\begin{aligned}
0 &= \binom{m}{q}\left[qp^{(q-1)}(1-p)^{(m-q)}-(m-q)p^q(1-p)^{(m-q-1)}\right] \\
& = \binom{m}{q}\left[p^{(q-1)}(1-p)^{(m-q-1)}(q-mp)\right]
\end{aligned}$$可求解得到$\displaystyle p=0,p=1,p=\frac{q}{m}$  
显然$\displaystyle P(X^{(j)}=a_{jl}|Y=c_k)=p=\frac{q}{m}=\frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k)}{\displaystyle \sum_{i=1}^N I(y_i=c_k)}$，公式(4.9)得证。

### 习题4.2
&emsp;&emsp;用贝叶斯估计法推出朴素贝叶斯法中的慨率估计公式(4.10)及公式(4.11)

**解答：**  
**第1步：**证明公式(4.11)：$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$  
加入先验概率，在没有任何信息的情况下，可以假设先验概率为均匀概率（即每个事件的概率是相同的）。  
可得$\displaystyle p=\frac{1}{K} \Leftrightarrow pK-1=0\quad(1)$  
根据习题4.1得出先验概率的极大似然估计是$\displaystyle pN - \sum_{i=1}^N I(y_i=c_k) = 0\quad(2)$  
存在参数$\lambda$使得$(1) \cdot \lambda + (2) = 0$  
所以有$$\lambda(pK-1) + pN - \sum_{i=1}^N I(y_i=c_k) = 0$$可得$\displaystyle P(Y=c_k) = \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}$，公式(4.11)得证。  

----

**第2步：**证明公式(4.10)：$\displaystyle P_{\lambda}(X^{(j)}=a_{jl} | Y = c_k) = \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}$   
根据第1步，可同理得到$$
P(Y=c_k, x^{(j)}=a_{j l})=\frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}$$  
$$\begin{aligned} 
P(x^{(j)}=a_{jl} | Y=c_k)
&= \frac{P(Y=c_k, x^{(j)}=a_{j l})}{P(y_i=c_k)} \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K \lambda}} \\
&= (\lambda可以任意取值，于是取\lambda = S_j \lambda) \\
&= \frac{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{N+K S_j \lambda}}{\displaystyle \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda}{N+K S_j \lambda}} \\ 
&= \frac{\displaystyle \sum_{i=1}^N I(y_i=c_k, x_i^{(j)}=a_{jl})+\lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + \lambda} (其中\lambda = S_j \lambda)\\
&= \frac{\displaystyle \sum_{i=1}^N I(x_i^{(j)}=a_{jl},y_i=c_k) + \lambda}{\displaystyle \sum_{i=1}^N I(y_i=c_k) + S_j \lambda}
\end{aligned} $$  
公式(4.11)得证。

----
参考代码：https://github.com/wzyonggege/statistical-learning-method

本文代码更新地址：https://github.com/fengdu78/lihang-code

习题解答：https://github.com/datawhalechina/statistical-learning-method-solutions-manual

中文注释制作：机器学习初学者公众号：ID:ai-start-com

配置环境：python 3.5+

代码全部测试通过。
![gongzhong](../gongzhong.jpg)